# Lesson 15 - Advanced construction and small types

This notebook supports the materials in [lesson 15 of C++ for Finance](https://loz-hurst.github.io/cpp-finance-materials-new/lessons/lesson-15.html).

## Constructors

Recall that constructors can be called explicitly or implicitly:

In [ ]:
class MyFirstClass {
public:
    MyFirstClass(int a) {}
    MyFirstClass(double a) {}
};

In [ ]:
MyFirstClass my_first_class_instance_1 {2}; // Explicit call

In [ ]:
MyFirstClass my_first_class_instance_2 = 2.2; // Implicit call

We can prevent implicit conversion by adding the keyword `explicit` to the constructor:

In [ ]:
class MySecondClass {
public:
    explicit MySecondClass(int a) {}
    explicit MySecondClass(double a) {}
};

In [ ]:
MySecondClass my_second_class_instance_1 {2}; // Explicit call - fine

In [ ]:
MySecondClass my_second_class_instance_2 = 2.2; // Implicit call - fail